In [1]:
import pandas as pd

# 缺失值处理

In [2]:
# 加载数据
train = pd.read_csv('./data/titanic_train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 缺失情况统计函数
def missing_values_table(df):
    # 计算所有的缺失值
    mis_val = df.isnull().sum()

    # 计算缺失值的比例
    mis_val_percent = 100 * mis_val / len(df)

    # 将结果拼接成 DataFrame
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # 将列重命名
    mis_val_table.columns = ['缺失值', '占比(%)']

    # 将缺失值为0的列去除，并按照缺失值占比进行排序
    mis_val_table_sorted = mis_val_table[mis_val_table['缺失值']!=0].sort_values(
        '占比(%)', ascending=False)

    # 打印信息
    print(f'传入的数据集共{df.shape[1]}列，\n其中{mis_val_table_sorted.shape[0]}列有缺失值')

    return mis_val_table_sorted

In [4]:
missing_values_table(train)

传入的数据集共12列，
其中3列有缺失值


,缺失值,占比(%)
Cabin,687,77.104377
Age,177,19.865320
Embarked,2,0.224467


## 删除方式

### 删除含有缺失值的行

**按行删除：删除指定列为缺失值的行记录，当数据记录比较多，而缺失的行占比较小时，可以考虑删除含有缺失值的行**

### 删除含有缺失值的列

**按列删除：当一列包含了很多缺失值的时候（比如超过80%），可以使用df.drop(['列名',..], axis=1)函数将指定列删除，但最好不要删除数据**

## 填充方式

### 非时序数据

#### 填充常量值

#### 填充统计值(缺失值所处列的平均值、中位数、众数)

### 时序数据

#### 前项填充

**前项填充：使用缺失值之前的第一个非缺失值进行填充**

#### 后项填充

**后项填充：使用缺失值之后的第一个非缺失值进行填充**

#### 线性插值填充

**线性插值填充：使用缺失值两端的非缺失值，建立线性关系，利用线性关系填充值**